In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA, KernelPCA
from sklearn.ensemble import RandomForestClassifier
import optuna
import optuna.visualization as vis

n = 9

X = np.load('Datasets/kryptonite-%s-X.npy'%(n))
y = np.load('Datasets/kryptonite-%s-y.npy'%(n))

def objective(trial):
    ### ========== Shuffle and Split Data ========== ###

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.8, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # Suggest dimensionality reduction and kernel parameters
    use_pca = trial.suggest_categorical('use_pca', ['True', 'False'])
    
    if use_pca == 'True':
        kernel_pca = trial.suggest_categorical('pca_kernel', ['linear', 'poly', 'rbf', 'sigmoid', 'cosine'])
        # kernel_pca = 'linear'

        params_pca = {}

        if kernel_pca == 'poly':
            params_pca['degree'] = trial.suggest_int('degree_pca', 1, 10)
        if kernel_pca in ['poly', 'rbf', 'sigmoid']:
            params_pca['gamma'] = trial.suggest_float('gamma_pca', 0.1, 10)

        params_pca['n_jobs'] = -1
        params_pca['n_components'] = trial.suggest_int('n_pca_components', 1, max(1, X_train_scaled.shape[1]))


    ### ========== Dimensionality Reduction ========== ###

    if use_pca == 'True':
        pca = KernelPCA(**params_pca)

        X_train_scaled = pca.fit_transform(X_train_scaled)
        X_val_scaled = pca.transform(X_val)

    # KNN Classifier parameters
    classifier = RandomForestClassifier(
        n_estimators=trial.suggest_int('n_estimators', 100, 300, step=25),
        n_jobs=-1
    )

    # Train the classifier
    classifier.fit(X_train_scaled, y_train)

    # Evaluate on the val set
    y_val_pred = classifier.predict(X_val_scaled)
    val_accuracy = accuracy_score(y_val, y_val_pred)

    # Report validation accuracy for pruning
    trial.report(val_accuracy, step=0)

    # Check if the trial should be pruned
    if trial.should_prune():
        raise optuna.TrialPruned()


    return val_accuracy

# Create a study object with a pruner
pruner = optuna.pruners.MedianPruner()
study = optuna.create_study(direction="maximize", pruner=pruner)

# Optimize the objective function
study.optimize(objective, n_trials=100)

print("Best hyperparameters:", study.best_params)
print("Best value:", study.best_value)

# Visualize the optimization history and parameter importances
vis.plot_optimization_history(study).show()
vis.plot_param_importances(study).show()
vis.plot_slice(study).show()

[I 2024-11-11 19:28:32,034] A new study created in memory with name: no-name-f578082f-1ddb-4143-a996-44dda66ac9fa
[I 2024-11-11 19:28:32,715] Trial 0 finished with value: 0.49895833333333334 and parameters: {'use_pca': 'True', 'n_pca_components': 1, 'pca_kernel': 'poly', 'n_estimators': 100, 'max_depth': 29, 'max_features': 'sqrt', 'criterion': 'log_loss'}. Best is trial 0 with value: 0.49895833333333334.


Test Accuracy: 0.5007


[I 2024-11-11 19:28:33,858] Trial 1 finished with value: 0.49166666666666664 and parameters: {'use_pca': 'True', 'n_pca_components': 2, 'pca_kernel': 'rbf', 'n_estimators': 300, 'max_depth': 24, 'max_features': 'log2', 'criterion': 'log_loss'}. Best is trial 0 with value: 0.49895833333333334.


Test Accuracy: 0.4905


[I 2024-11-11 19:28:34,322] Trial 2 finished with value: 0.5100694444444445 and parameters: {'use_pca': 'False', 'n_pca_components': 2, 'pca_kernel': 'cosine', 'n_estimators': 175, 'max_depth': 41, 'max_features': 'log2', 'criterion': 'entropy'}. Best is trial 2 with value: 0.5100694444444445.


...

Test Accuracy: 0.7135


[I 2024-11-11 19:30:02,143] Trial 95 finished with value: 0.6982638888888889 and parameters: {'use_pca': 'True', 'n_pca_components': 9, 'pca_kernel': 'rbf', 'n_estimators': 275, 'max_depth': 28, 'max_features': 'sqrt', 'criterion': 'entropy'}. Best is trial 57 with value: 0.7135416666666666.


Test Accuracy: 0.7017


[I 2024-11-11 19:30:03,161] Trial 96 pruned. 
[I 2024-11-11 19:30:04,221] Trial 97 pruned. 
[I 2024-11-11 19:30:05,172] Trial 98 pruned. 
[I 2024-11-11 19:30:06,054] Trial 99 pruned. 


Best hyperparameters: {'use_pca': 'True', 'n_pca_components': 9, 'pca_kernel': 'rbf', 'n_estimators': 225, 'max_depth': 27, 'max_features': 'sqrt', 'criterion': 'entropy'}
Best value: 0.7135416666666666
